# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy.engine import create_engine

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df['message'].values
y = df.iloc[:, 4:].values
category_names = list(df.iloc[:, 4:].columns)

### 2. Write a tokenization function to process your text data

In [4]:
import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [51]:
import re
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

def tokenize(text):
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize text
    tokens = word_tokenize(text)
    
    # Remove stop words
    tokens = [t for t in tokens if t not in stopwords.words("english")]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    # reduce words to their root form using default pos
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    # lemmatize verbs by specifying pos
    tokens = [lemmatizer.lemmatize(t, pos='v') for t in tokens]
    # lemmatize verbs by specifying pos
    tokens = [lemmatizer.lemmatize(t, pos='a') for t in tokens]
    
    # Stemming
    tokens = [PorterStemmer().stem(t) for t in tokens]

    # drop duplicates 
    tokens = pd.Series(tokens, dtype='object').drop_duplicates().tolist()
    
    return tokens

In [52]:
t = 'Weather update - a cold front from Cuba that could pass over Haiti weathers'
tokenize(t)

['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pa', 'haiti']

In [7]:
i=4
X[i], tokenize(X[i])

('says: west side of Haiti, rest of the country today and tonight',
 ['say', 'west', 'side', 'haiti', 'rest', 'countri', 'today', 'tonight'])

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [44]:
# build pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from models.custom_transformer import StartingVerbExtractor
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC

pipeline = Pipeline([('features', FeatureUnion([('text_pipeline', Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                                                                            ('tfidf', TfidfTransformer())])),
                                                ('starting_verb', StartingVerbExtractor())])),
                     ('clf', MultiOutputClassifier(LinearSVC()))])

In [19]:
X[2]

'Looking for someone but no name'

In [20]:
StartingVerbExtractor().transform(X[2])

,0
0,True


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [53]:
# Split data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

# train classifier
# measure training time
import time
start_time = time.time()

pipeline.fit(X_train, y_train)

train_time = time.time() - start_time

print("--- %s minutes ---" % (train_time/60))

C:\Users\ADMIN\AppData\Local\Temp/ipykernel_544/2230505817.py:30: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tokens = pd.Series(tokens).drop_duplicates().tolist()


--- 7.794838774204254 minutes ---


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [22]:
X_test[0]

'But as winter rains lash the area and lightning flashes across the night skies, thousands continue to lie in the open.'

In [23]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [24]:
y_pred.shape

(6554, 35)

In [25]:
y_pred[0]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0], dtype=int64)

In [26]:
# display results
from sklearn.metrics import classification_report

colnames = list(df.iloc[:, 4:].columns)
for k in range(len(colnames)):
    print(k, '. ', colnames[k], '. \t acc = ', (y_pred[:100, k] == y_test[:100,k]).mean())
    print(classification_report(y_test[:100,k], y_pred[:100,k]))

0 .  related . 	 acc =  0.83
              precision    recall  f1-score   support

           0       0.70      0.56      0.62        25
           1       0.86      0.92      0.89        75

    accuracy                           0.83       100
   macro avg       0.78      0.74      0.76       100
weighted avg       0.82      0.83      0.82       100

1 .  request . 	 acc =  0.93
              precision    recall  f1-score   support

           0       0.93      0.99      0.96        83
           1       0.92      0.65      0.76        17

    accuracy                           0.93       100
   macro avg       0.92      0.82      0.86       100
weighted avg       0.93      0.93      0.92       100

2 .  offer . 	 acc =  1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       100

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00 

C:\Users\ADMIN\anaconda3\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ADMIN\anaconda3\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ADMIN\anaconda3\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ADMIN\anaconda3\envs\m

              precision    recall  f1-score   support

           0       0.99      1.00      0.99        99
           1       0.00      0.00      0.00         1

    accuracy                           0.99       100
   macro avg       0.49      0.50      0.50       100
weighted avg       0.98      0.99      0.99       100

15 .  refugees . 	 acc =  0.98
              precision    recall  f1-score   support

           0       0.99      0.99      0.99        97
           1       0.67      0.67      0.67         3

    accuracy                           0.98       100
   macro avg       0.83      0.83      0.83       100
weighted avg       0.98      0.98      0.98       100

16 .  death . 	 acc =  0.98
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        94
           1       1.00      0.67      0.80         6

    accuracy                           0.98       100
   macro avg       0.99      0.83      0.89       100
weighted avg   

C:\Users\ADMIN\anaconda3\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ADMIN\anaconda3\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ADMIN\anaconda3\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
    'features__text_pipeline__vect__max_df': (0.75, 1.0),
    'features__text_pipeline__vect__max_features': (None, 1000, 2000),
    'features__text_pipeline__tfidf__use_idf': (True, False),
    'clf__n_estimators': [20, 50, 100],
    'clf__min_samples_split': [2, 3, 4],
    'features__transformer_weights': (
        {'text_pipeline': 1, 'starting_verb': 0.5},
        {'text_pipeline': 0.5, 'starting_verb': 1},
        {'text_pipeline': 0.8, 'starting_verb': 1},
    )
}

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = model.predict(X_test)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [31]:
pipeline.get_params()

{'memory': None,
 'steps': [('features',
   FeatureUnion(transformer_list=[('text_pipeline',
                                   Pipeline(steps=[('vect',
                                                    CountVectorizer(tokenizer=<function tokenize at 0x0000015C87A325E0>)),
                                                   ('tfidf',
                                                    TfidfTransformer())])),
                                  ('starting_verb', StartingVerbExtractor())])),
  ('clf', MultiOutputClassifier(estimator=LinearSVC()))],
 'verbose': False,
 'features': FeatureUnion(transformer_list=[('text_pipeline',
                                 Pipeline(steps=[('vect',
                                                  CountVectorizer(tokenizer=<function tokenize at 0x0000015C87A325E0>)),
                                                 ('tfidf',
                                                  TfidfTransformer())])),
                                ('starting_verb', Start

In [34]:
import pickle
filename = 'data/classifier_v1.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.